In [2]:
# basic imports
import numpy as np
import pandas as pd

# demos for visualization
from nilearn import datasets
from nilearn import plotting
from nilearn.connectome import GroupSparseCovariance
from nilearn.connectome import ConnectivityMeasure

# for ML
from sklearn.metrics import mutual_info_score

# os library
from os.path import exists, join

C:\Users\simon\Anaconda3\lib\site-packages\nilearn\datasets\__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [3]:
data_path = join(".", "data") if exists(join(".", "data")) \
    else join("..", "input", "statistical-learning-sapienza-spring-2021")

In [4]:
train = pd.read_csv(join(data_path, 'train.csv'))
test = pd.read_csv(join(data_path, 'test.csv'))

In [5]:
len(train.id.values)
len(train.index.values)

600

In [6]:
def get_data(entry):
    id = entry.id
    variables = entry[['var1', 'var2', 'var3']].to_numpy()
    timeseries = entry[5 if "y" in entry.index else 4:].to_numpy(dtype=float).reshape((115, 116), order="F")
    if "y" in entry.index: return id, entry.y, variables, timeseries
    else: return id, variables, timeseries

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [297]:
time_series = train.apply(lambda row: get_data(row)[-1], axis=1).to_list()

In [298]:
time_series_test = test.apply(lambda row: get_data(row)[-1], axis=1).to_list()




In [299]:
time_series = list(map(lambda ts: (np.diff(ts, axis=0) - np.diff(ts, axis=0).min(axis=0)) / np.mean(np.diff(ts, axis=0).max(axis=0) - np.diff(ts, axis=0).min(axis=0)), time_series))
time_series_test = list(map(lambda ts: (np.diff(ts, axis=0) - np.diff(ts, axis=0).min(axis=0)) / np.mean(np.diff(ts, axis=0).max(axis=0) - np.diff(ts, axis=0).min(axis=0)), time_series_test))


In [300]:
def binning(ts):        
    binned_ts = np.array_split(ts, 11, axis = 0)
    return np.concatenate([list(map(lambda x: x.max(axis=0), binned_ts)), list(map(lambda x: x.min(axis=0), binned_ts))], axis = 1)

time_series = list(map(binning, time_series))
time_series_test = list(map(binning, time_series_test))
    

In [292]:
# import nitime.timeseries as ts
# time_series = list(map(lambda x: ts.TimeSeries(x.T, sampling_interval=1), time_series))
# time_series_test = list(map(lambda x: ts.TimeSeries(x.T, sampling_interval=1), time_series_test))

In [ ]:
# time_series_test[0].shape

In [ ]:
# import nitime.analysis as nta
# import time
# start = time.time()
# f_ub = 0.15
# f_lb = 0.02
# causalities = []
# for i, x in enumerate(time_series):
#     if i%60 == 0:
#         print(f'{i/6}% done in {round(time.time() - start, 2)} seconds')
#     G=nta.GrangerAnalyzer(x, order=3)
#     freq_idx_G = np.where((G.frequencies > f_lb) * (G.frequencies < f_ub))[0]
#     g1 = np.mean(G.causality_xy[:, :, freq_idx_G], -1)
#     causalities.append(g1)
# print(f'Granger Analysis done in {round((time.time() - start)/60, 2)} minutes')

In [301]:
def apply_connectivity(time_series, vect=True, diag=True):

    correlation_measure = ConnectivityMeasure(kind='partial correlation',
                                              vectorize = vect,
                                              discard_diagonal=diag)

    return correlation_measure.fit_transform(time_series)

In [302]:
correlation_matrices = apply_connectivity(time_series)
correlation_test = apply_connectivity(time_series_test)

In [303]:
correlation_matrices

array([[-3.65429937e-04, -6.21004741e-03, -5.70190950e-04, ...,
        -1.76604213e-02,  7.72765761e-03,  3.29291850e-02],
       [-4.91986566e-04, -5.55966565e-05, -4.38985714e-04, ...,
         2.75550631e-02,  1.06291358e-02,  2.06050232e-03],
       [ 6.14522091e-03, -2.50620666e-03, -4.02427778e-03, ...,
        -1.10529348e-02, -6.55763940e-03,  1.77536219e-02],
       ...,
       [ 1.33235653e-03, -6.68156820e-03,  1.16697641e-03, ...,
        -4.74873626e-02,  2.03675157e-02,  3.75460637e-02],
       [-1.41345640e-03, -3.51419225e-03,  2.81724477e-03, ...,
        -6.36356522e-02,  2.06931914e-02,  7.23850604e-02],
       [ 3.40702685e-04,  9.26106420e-03,  1.60163611e-03, ...,
        -1.20350223e-02,  3.08039435e-03, -6.81006409e-03]])

In [304]:
pd.concat([train.iloc[:, :5], pd.DataFrame(correlation_matrices)], axis=1).to_csv('train_corr.csv')
pd.concat([test.iloc[:, :4], pd.DataFrame(correlation_test)], axis=1).to_csv('test_corr.csv')

In [36]:
pd.concat([test.iloc[:, :4], pd.DataFrame(correlation_test)], axis=1)

,id,var1,var2,var3,0,1,2,3,4,5,...,6660,6661,6662,6663,6664,6665,6666,6667,6668,6669
0,17,32,A,A,0.023671,-0.007289,-0.009024,0.007298,-0.003480,0.000240,...,0.063989,-0.048858,0.054061,-0.228254,-0.062239,0.001865,0.103494,-0.044311,0.101675,-0.007015
1,19,11,C,A,0.026247,0.023898,-0.039581,-0.010329,-0.016104,-0.005682,...,0.086398,0.191093,0.095374,-0.209788,0.026410,-0.056142,0.075483,0.030596,-0.038504,-0.118798
2,25,17,A,C,0.011462,-0.002216,0.007523,-0.003675,0.020650,0.006799,...,0.066654,0.005996,0.099345,0.158647,0.004112,0.244222,-0.036526,-0.177120,0.026056,-0.026664
3,32,17,A,C,-0.000732,-0.006386,-0.020390,-0.014332,0.017277,0.001105,...,0.087192,-0.120249,-0.001533,0.030498,0.177132,-0.031438,0.083403,0.104198,0.042981,0.032002
4,33,16,A,C,0.055043,-0.020623,-0.046023,0.012963,-0.024777,0.064455,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,777,20,A,C,-0.003031,0.007281,-0.024983,-0.017637,0.044618,-0.008667,...,0.140488,-0.114554,-0.043859,0.306518,-0.035661,0.032000,-0.014286,-0.049970,0.145010,0.239272
195,783,24,A,C,0.012482,0.003675,-0.025090,0.010962,0.035677,0.012848,...,0.246349,0.081832,0.060359,-0.243860,-0.027795,0.073448,-0.095433,0.182376,-0.182050,0.381747
196,784,24,A,C,0.059117,-0.019442,0.004688,-0.003915,0.041571,-0.025616,...,0.288217,0.174798,0.001413,0.005653,-0.174478,0.037918,-0.085111,0.042151,-0.294607,0.225792
197,792,29,C,C,0.019908,-0.000896,-0.017489,-0.028130,0.023437,0.000375,...,0.248101,-0.018863,0.092836,0.503031,-0.266960,0.069081,0.307400,-0.216884,0.281473,0.112249


In [40]:
y_train = train.loc[:, 'y']
X_train = train.drop(columns=['y'])

(600, 13344)

In [55]:
from sklearn.decomposition import KernelPCA
transformer = KernelPCA(kernel='rbf')
X_transformed = transformer.fit_transform(correlation_matrices)
X_transformed.shape

(600, 599)

In [56]:
X_trans_test = transformer.transform(correlation_test)

In [57]:
pd.concat([X_train.iloc[:, :4], y_train, pd.DataFrame(X_transformed)], axis=1).to_csv('train_pca.csv', index=False)
pd.concat([test.iloc[:, :4], pd.DataFrame(X_trans_test)], axis=1).to_csv('test_pca.csv', index=False)
